In [1]:
from transformers import AutoTokenizer

# Load the tokenizer for the Phi-3-mini model
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

# Example input text
input_text = "Task is stack BoxI on top of BallJ\nYou are a robotic arm with advanced planning capabilities. Your task is to generate Python code using parameterized skills (open_gripper(), close_gripper(), move_to_position(), get_graspable_point(), get_size()) that accomplishes the user's specified task.\nPlease produce executable Python code that employs these pre-scripted parameterized skills. Remember to import the necessary package before running the code. Carefully think through your plans and code.\nWhen generating plans, consider spatial relationships meticulously. \nFor example: If you need to pick up an object, first move to a position above it, then move down to grasp it. Moving directly to the object's position may push it away. Treat it as a two-step process. After this, consider whether the gripper might hit another object while moving to the next position.\nHere is an example snippet for your reference, demonstrating how to call the function:\n\"\"\npython\nimport numpy as np  # import numpy because we are using it below\nopen_gripper()\nclose_gripper()\n# Get the graspable point of cubeA\ncubeA_graspable_point = get_graspable_point('cubeA')\n# Get size of cubeA\ncubeA_size = get_size('cubeA')\nmove_to_position(cubeA_graspable_point)\n\"\"\nPlease generate a step-by-step plan followed by a single Python code block. \nYou don't need to define all the functions again, just use them. There's no need to define a function for the task, just generate the code.<|end|>\n<|assistant|>\n**Plan:**\n1. Open the gripper to ensure it is ready to grasp objects.\n2. Get the graspable point and size of BoxI.\n3. Move to a position directly above BoxI.\n4. Move down to the graspable point of BoxI.\n5. Close the gripper to grasp BoxI.\n6. Get the graspable point and size of BallJ.\n7. Move to a position directly above BallJ while ensuring BoxI does not collide with BallJ.\n8. Attempt to move down to place BoxI on top of BallJ. This will fail as the ball is not a stable surface for the box.\n9. Open the gripper to release BoxI.\n10. Move the gripper away from BallJ to complete the task.\n**Python Code:**\npython\nimport numpy as np  # Import numpy because we are using it below\n# Step 1: Open the gripper\nopen_gripper()\n# Step 2: Get the graspable point and size of BoxI\nboxI_graspable_point = get_graspable_point('BoxI')\nboxI_size = get_size('BoxI')\n# Step 3: Move to a position directly above BoxI\nabove_boxI = boxI_graspable_point + np.array([0, 0, boxI_size[2] + 0.1])  # Adding a buffer of 0.1 units above BoxI\nmove_to_position(above_boxI)\n# Step 4: Move down to the graspable point of BoxI\nmove_to_position(boxI_graspable_point)\n# Step 5: Close the gripper to grasp BoxI\nclose_gripper()\n# Step 6: Get the graspable point and size of BallJ\nballJ_graspable_point = get_graspable_point('BallJ')\nballJ_size = get_size('BallJ')\n# Step 7: Move to a position directly above BallJ\nabove_ballJ = ballJ_graspable_point + np.array([0, 0, ballJ_size[2] + boxI_size[2] + 0.1])  # Adding a buffer of 0.1 units above BallJ\nmove_to_position(above_ballJ)\n# Step 8: Attempt to move down to place BoxI on top of BallJ (this will fail)\njust_above_ballJ = ballJ_graspable_point + np.array([0, 0, ballJ_size[2] + boxI_size[2]])\nmove_to_position(just_above_ballJ)\n# Step 9: Open the gripper to release BoxI\nopen_gripper()\n# Step 10: Move the gripper away from BallJ to complete the task\nmove_to_position(above_ballJ)\n"

# Tokenize the input text
tokens = tokenizer.tokenize(input_text)

# Print the number of tokens
print(f"Number of tokens: {len(tokens)}")

# Optionally, print the tokens themselves
print(tokens)


/home/xilun/miniconda3/envs/et_robot/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Number of tokens: 1049
['▁Task', '▁is', '▁stack', '▁Box', 'I', '▁on', '▁top', '▁of', '▁Ball', 'J', '<0x0A>', 'You', '▁are', '▁a', '▁robot', 'ic', '▁arm', '▁with', '▁advanced', '▁planning', '▁capabilities', '.', '▁Your', '▁task', '▁is', '▁to', '▁generate', '▁Python', '▁code', '▁using', '▁parameter', 'ized', '▁skills', '▁(', 'open', '_', 'g', 'ri', 'pper', '(),', '▁close', '_', 'g', 'ri', 'pper', '(),', '▁move', '_', 'to', '_', 'position', '(),', '▁get', '_', 'gr', 'asp', 'able', '_', 'point', '(),', '▁get', '_', 'size', '())', '▁that', '▁accomplish', 'es', '▁the', '▁user', "'", 's', '▁specified', '▁task', '.', '<0x0A>', 'Please', '▁produce', '▁executable', '▁Python', '▁code', '▁that', '▁emp', 'lo', 'ys', '▁these', '▁pre', '-', 'script', 'ed', '▁parameter', 'ized', '▁skills', '.', '▁Remember', '▁to', '▁import', '▁the', '▁necessary', '▁package', '▁before', '▁running', '▁the', '▁code', '.', '▁Care', 'fully', '▁think', '▁through', '▁your', '▁plans', '▁and', '▁code', '.', '<0x0A>', 'When', '

## Distributed Fine Tuning


In [12]:
import os 
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel, PeftConfig
import json

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.random.manual_seed(0)

# Ensure the device is set correctly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the directory where the fine-tuned LoRA adapters are saved
output_dir = "phi-3-mini-LoRA/checkpoint-725/"

model_name = "microsoft/Phi-3-mini-4k-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(output_dir, trust_remote_code=True, add_eos_token=True, use_fast=True)


# Load the LoRA adapter configuration
peft_config = PeftConfig.from_pretrained(output_dir)

# Apply the LoRA adapter to the base model
# model = PeftModel.from_pretrained(model, output_dir, torch_dtype=torch.bfloat16)
# model = model.merge_and_unload()
model = model.to(device)

# prompt_path = '/home/xilun/ET_robot/prompt_stack_two.txt'
prompt_path = "/home/xilun/ET_robot/dataset/172_dataset.json"
# Load the prompt as json file
prompt = json.loads(open(prompt_path).read())
prompt = prompt[0]["input"]
## delete <|user|> and <|assistant|> and <|end|> if found in prompt 
prompt = prompt.replace("Starts<", "").replace(">Ends", "")
prompt = prompt.replace("<|user|>", "").replace("<|assistant|>", "").replace("<|end|>", "")
print(prompt)
# inputs = tokenizer(prompt, return_tensors="pt").to(device)
# print (inputs)
# input()
# prompt = prompt + "<|assistant|>"
messages = [
    {"role": "user", "content": prompt},
]

# Generate text using the model's generate method
generation_args = {
    "max_new_tokens": 1500,
    "temperature": 0.2,
    "do_sample": False,
    "return_full_text": False,
    # "top_k": 50,
    # "top_p": 0.95
    
}

# Generate the output
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])



# output = model.generate(
#     input_ids=inputs["input_ids"],
#     max_new_tokens=generation_args["max_new_tokens"],
#     temperature=generation_args["temperature"],
#     do_sample=generation_args["do_sample"],
#     # top_k=generation_args["top_k"],
#     # top_p=generation_args["top_p"],
#     # pad_token_id=tokenizer.pad_token_id,
#     # eos_token_id=tokenizer.eos_token_id,
# )
# # Decode the generated output with chat template



# output_text = tokenizer.decode(output[0], skip_special_tokens=True)
# print(output_text)
del model

# 'import gc' is used to import Python's garbage collector module.
import gc

# 'gc.collect()' is a method that triggers a full garbage collection, which can help to free up memory.
# It's called twice here to ensure that all unreachable objects are collected.
gc.collect()
gc.collect()
torch.cuda.empty_cache()


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.47it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.




Task is place CubeA on top of CubeB

You are a robotic arm with advanced planning capabilities. Your task is to generate Python code using parameterized skills (open_gripper(), close_gripper(), move_to_position(), get_graspable_point(), get_size()) that accomplishes the user's specified task.
Please produce executable Python code that employs these pre-scripted parameterized skills. Remember to import the necessary package before running the code. Carefully think through your plans and code.
When generating plans, consider spatial relationships meticulously. 
For example: If you need to pick up an object, first move to a position above it, then move down to grasp it. Moving directly to the object's position may push it away. Treat it as a two-step process. After this, consider whether the gripper might hit another object while moving to the next position.
Here is an example snippet for your reference, demonstrating how to call the function:
"""
python
import numpy as np  # import num

In [13]:
prompt_path = "/home/xilun/ET_robot/dataset/172_dataset.json"
# Load the prompt as json file
prompt = json.loads(open(prompt_path).read())
prompt = prompt[0]["input"]

print (prompt)

<|user|>Starts<

Task is place CubeA on top of CubeB

You are a robotic arm with advanced planning capabilities. Your task is to generate Python code using parameterized skills (open_gripper(), close_gripper(), move_to_position(), get_graspable_point(), get_size()) that accomplishes the user's specified task.
Please produce executable Python code that employs these pre-scripted parameterized skills. Remember to import the necessary package before running the code. Carefully think through your plans and code.
When generating plans, consider spatial relationships meticulously. 
For example: If you need to pick up an object, first move to a position above it, then move down to grasp it. Moving directly to the object's position may push it away. Treat it as a two-step process. After this, consider whether the gripper might hit another object while moving to the next position.
Here is an example snippet for your reference, demonstrating how to call the function:
"""
python
import numpy as n

In [ ]:
import os 
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel, PeftConfig, AutoPeftModelForCausalLM
import json

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.random.manual_seed(0)

# Ensure the device is set correctly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the directory where the fine-tuned LoRA adapters are saved
output_dir = "phi-3-mini-LoRA/checkpoint-1674/"
tokenizer = AutoTokenizer.from_pretrained(output_dir, trust_remote_code=True, add_eos_token=True, use_fast=True)

new_model = AutoPeftModelForCausalLM.from_pretrained(
    output_dir,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

new_model = new_model.to(device)
new_model = new_model.merge_and_unload()
# prompt_path = '/home/xilun/ET_robot/prompt_stack_two.txt'
prompt_path = "/home/xilun/ET_robot/dataset/172_dataset.json"
# Load the prompt as json file
prompt = json.loads(open(prompt_path).read())
prompt = prompt[0]["input"]
## delete <|user|> and <|assistant|> and <|end|> if found in prompt 
prompt = prompt.replace("<|user|>", "").replace("<|assistant|>", "").replace("<|end|>", "")


# Generate text using the model's generate method
generation_args = {
    "max_new_tokens": 1500,
    "temperature": 0,
    "do_sample": False,
    "return_full_text": True,
    # "top_k": 50,
    # "top_p": 0.95
    
}

# Generate the output
pipe = pipeline(
    "text-generation",
    model=new_model,
    tokenizer=tokenizer,
)

prompt = pipe.tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)


outputs = pipe(prompt, max_new_tokens=1500, do_sample=True, num_beams=1, temperature=0.3, top_k=50, top_p=0.95,
                   max_time= 180) #, eos_token_id=eos_token)
print (outputs[0]['generated_text'][len(prompt):].strip())



